In [1]:
import ipywidgets as widgets
import serial

In [175]:
ser = serial.Serial('COM3',9600, timeout=10)

In [167]:
commands=[]

In [173]:
baseSlider=widgets.IntSlider(min=-180,max=180,step=1,description='Base:',value=0)
shoulderSlider=widgets.IntSlider(min=-90,max=90,step=1,description='Shoulder:',value=0)
elbowSlider=widgets.IntSlider(min=-180,max=180,step=1,description='Elbow:',value=0)
wrist1Slider=widgets.IntSlider(min=-180,max=180,step=1,description='Wrist1:',value=0)
wrist2Slider=widgets.IntSlider(min=-180,max=180,step=1,description='Wrist2:',value=0)
wrist3Slider=widgets.IntSlider(min=-180,max=180,step=1,description='Wrist3:',value=0)
addToCommands = widgets.Button(description='Add')
sendLive = widgets.Checkbox(value=False,description='Send Live',disabled=False,indent=False)
setHome = widgets.Button(description='Set As Home')
commandDisp = widgets.Select(options=commands, description='Commands:')
removeLine = widgets.Button(description='Remove Line')
sendAll = widgets.Button(description='Send All')
wrist2KeepLevel = widgets.Checkbox(value=False,description="Keep Level",disabled=False, indent=False)

grid = widgets.GridspecLayout(6, 2,  height='300px')
grid[0,0] = baseSlider
grid[1,0] = shoulderSlider
grid[2,0] = elbowSlider
grid[3,0] = addToCommands
grid[4,:] = commandDisp
grid[5,0] = removeLine
grid[0,1] = wrist1Slider
grid[1,1] = widgets.HBox([wrist2Slider,wrist2KeepLevel])
grid[2,1] = wrist3Slider
grid[3,1] = widgets.HBox([sendLive,setHome])
grid[5,1] = sendAll
display(grid)

def makeCommand():
    command = '0{} 1{} 2{} 3{} 4{} 5{}\n'.format(baseSlider.value,shoulderSlider.value,elbowSlider.value,wrist1Slider.value,wrist2Slider.value,wrist3Slider.value)
    return command

def addToCommands_eventhandler(obj):
    commands.append(makeCommand())
    commandDisp.options=commands
    
addToCommands.on_click(addToCommands_eventhandler)

def setHome_eventhandler(obj):
    ser.write(b's/n')
    baseSlider.value=0
    shoulderSlider.value=0
    elbowSlider.value=0
    wrist1Slider.value=0
    wrist2Slider.value=0
    wrist3Slider.value=0
setHome.on_click(setHome_eventhandler)

originalWrist2 = 1

def keepLevelObserver(change):
    global originalWrist2
    if wrist2KeepLevel.value:
        originalWrist2 = wrist2Slider.value+shoulderSlider.value+elbowSlider.value
        
wrist2KeepLevel.observe(keepLevelObserver, names="value")
        
def coordinateObserver(change):
    if wrist2KeepLevel.value:
        wrist2Slider.value=originalWrist2-shoulderSlider.value-elbowSlider.value
    if sendLive.value:
        ser.write(makeCommand().encode('ASCII'))
    
baseSlider.observe(coordinateObserver,names="value")
shoulderSlider.observe(coordinateObserver,names="value")
elbowSlider.observe(coordinateObserver,names="value")
wrist1Slider.observe(coordinateObserver,names="value")
wrist2Slider.observe(coordinateObserver,names="value")
wrist3Slider.observe(coordinateObserver,names="value")

def removeLine_eventhandler(obj):
    commands.pop(commandDisp.index)
    commandDisp.options=commands

removeLine.on_click(removeLine_eventhandler)

def sendAll_eventhandler(obj):
    ser.reset_input_buffer()
    for command in commands:
        print("::" + command + "::")
        ser.reset_input_buffer()
        ser.write(command.encode('ASCII'))
        print(ser.readline())

sendAll.on_click(sendAll_eventhandler)




GridspecLayout(children=(IntSlider(value=0, description='Base:', layout=Layout(grid_area='widget001'), max=180…

In [174]:
ser.close() 

In [27]:
commandDisp.index

3